# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

## Generowanie kluczy w kryptosystemie RSA

### 1. Losujemy dwie duże liczby pierwsze 
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu. 
#### Skąd wziąć liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**


Test probabilistyczny, np. Rabina-Millera. **(A to już znamy!!!)**

## Zadanie
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

In [7]:
import random
import math

In [8]:
def prime_naive(n):
    if n < 2:
        return False

    m = math.isqrt(n)
    for i in range(2, m + 1):
        if n % i == 0:
            return False

    return True

In [9]:
SIZE = 1000001
IS_PRIME = [prime_naive(n) for n in range(SIZE)]
PRIMES = [n for n in range(SIZE) if IS_PRIME[n]]

In [10]:
def check_composite(n: int, a: int, d: int, s: int) -> bool:
    x = pow(a, d, mod=n)
    if x == 1 or x == n - 1:
        return False
    for r in range(1, s):
        x = x * x % n
        if x == n - 1:
            return False
    return True

In [11]:
def miller_rabin_test(n: int, number_of_iter: int = 1) -> bool:
    if n < 4:
        return n == 2 or n == 3
    
    s, d = 0, n - 1
    while (d & 1) == 0:
        d >>= 1
        s += 1
    
    for i in range(number_of_iter):
        a = random.randint(2, n - 2)
        if check_composite(n, a, d, s):
            return False
    return True

In [23]:
#napisz funkcję generującą liczbę pierwszą 
def generatePrime(keysize):
    #napisz swój kod tutaj
    while True:
        bignum = random.getrandbits(keysize)
        bignum |= (1 << (keysize - 1)) | 1
        if miller_rabin_test(bignum, number_of_iter=5):
            return bignum

### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$ $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1)\cdot (q-1)$ (czasami używane jest w miejscu pojęcie tocjentu lub funkcji Eulera: $\phi(n) = \phi(p)\cdot \phi(q) = (p − 1)·(q − 1)$)
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)\cdot (q-1))$: $de \equiv 1  (mod\ (p-1)\cdot (q-1))$ **(potrzebujemy rozszerzonego algorytmu Euklidesa!!!)**
5. Kluczem publiczny jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

In [13]:
def extended_gcd(a: int, b: int):
    x, y = 1, 0
    x1, y1 = 0, 1
    while b:
        q = a // b
        x, x1 = x1, x - q * x1
        y, y1 = y1, y - q * y1
        a, b = b, a - q * b
    return x, y, a

In [14]:
def modulo_inverse(a: int, mod: int) -> int:
    x, _, g = extended_gcd(a, mod)
    return x if g == 1 else None

## Zadanie 

1. Napisz funkcję generującą klucze RSA o ustalonym rozmiarze

In [58]:
import random, sys, os

def generateKey(keySize):
    #napisz swój kod tutaj
    p, q = generatePrime(keySize), generatePrime(keySize)
    phi, n = (p - 1) * (q - 1), p * q
    e = random.randint(3, phi - 1)
    while extended_gcd(e, phi)[2] != 1:
        e = random.randint(3, phi - 1)
    d = modulo_inverse(e, phi)
    publicKey = (e, n)
    privateKey = (n, d)
    return (publicKey, privateKey)

def makeKeyFiles(keySize):
    public, private = generateKey(keySize)
  
print('Generujemy klucze publiczny i prywatny')
makeKeyFiles(16)

Generujemy klucze publiczny i prywatny


## Zadanie 

Napisz funkcje implementujące szyfrowanie i deszyfrowanie RSA (tzw. podręcznikowe)

### Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [32]:
def encrypt(message, modulus, exp):
    # kod szyfrowania     
    message_encrypted = list(map(lambda m : pow(ord(m), exp, modulus), list(message)))
    return message_encrypted

### Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [66]:
def decrypt(message_encrypted, modulus, exp):
    message_ascii = list(map(lambda c : chr(pow(c, exp, modulus)), message_encrypted))
    return ('').join(message_ascii)

In [60]:
public, private = generateKey(16)

In [64]:
cipher = encrypt("Hello world!", public[1], public[0])
print(cipher)

[2682023312, 398679798, 250630766, 250630766, 1550109001, 1573117807, 2118358109, 1550109001, 1718714481, 250630766, 1700197442, 264252113]


In [67]:
message = decrypt(cipher, private[0], private[1])
print(message)

Hello world!


## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy) - zweryfikuj jak na wydajnosć wpływa zastosowanie różnych sposobów potęgowania dostępnych w Python i jego bibliotekach.  
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 


### Algorytm szybkiego potęgowania 
1. Zwykłe potęgowanie $n^{exp}$: $exp$ mnożeń 
2. Algorytm szybkiego potęgowania: część mnożeń zastępujemy podnoszeniem do kwadratu (_squaring_).
    __Skąd mamy wiedzieć kiedy mnożyć, a kiedy potęgować?__

In [27]:

def fastModularExponentation(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m